## XML-Struktur
fra Ellens mail

All kildetekst ligger innenfor `<div type="chapter" subtype="source">`.
Alle innledninger og annen utgivertekst ligger i `<div type="chapter" subtype="intro">`, disse kan nok undertrykkes. Det samme gjelder illustrasjoner som er kodet i `<div type="chapter" subtype="illustration">`
 
Innenfor div ligger kildetekst hovedsakelig i `<head>, <p> og/eller <lg>/<l>`.  Og innenfor alle disse kan det forekomme spesialkoder som `<dateline>, <signed>, <del>, <add>, <unclear>, <gap/>, <note>, <pb/>, <hi> og <emph>`.  Noen ganger forekommer underkapitler, under-<div>’er, som også kan inneholde alt det nevnte. Så hvis du har en kommando som sier ta med all tekst innenfor en `<div type="chapter" subtype="source">` uavhengig av «child»/undernivå hadde nok det funket fint…..
 
Når det gjelder metadata, så kan du nok bruke `<titleStmt>` som ligger i `<fileDesc> i <teiHeader>` til noe, kodingen i `<titleStmt>` ser slik ut:
```<titleStmt>
<title>Brevveksling 1874&#x2013;1908</title>
<author>Kitty L. Kielland og Eilif Peterssen</author>
<editor>Utgitt ved Anne Melg&#x00E5;rd, NB kilder 8:2, Nasjonalbiblioteket, Oslo 2019</editor>
</titleStmt>```

In [1]:
import dhlab.nbtext as nb
from dhlab.nbtokenizer import tokenize
from collections import Counter
import pandas as pd

In [2]:
import zipfile
from bs4 import BeautifulSoup
import sys
import zipfile
import re

In [3]:
def get_files_from_zip(document):
    """Find all documents in .zip"""
    import sys
    import zipfile
    import re
    from bs4 import BeautifulSoup

    with zipfile.ZipFile(document, 'r') as zfp:
           n = zfp.namelist()
    return n

In [4]:
def file_contents(name, zipfolder):
    
    with zipfile.ZipFile(zipfolder, 'r') as zfp:
        try:
            with zfp.open(name) as fp:
                soup = fp.read()
        except:
            print('problemer', name)
    return soup

In [125]:
def extract_text(file, zipfolder):
    text = []
    soup = BeautifulSoup(file_contents(file, zipfolder), 'xml')
    for node in soup.find_all('div', {"type":"chapter", "subtype":"source"}):
        for p in node.find_all(['p', 'head']) or p in node_fi:
            text.append(tokenize(p.text))
    return text

def extract_text_as_string(file, zipfolder):
    text = []
    soup = BeautifulSoup(file_contents(file, zipfolder), 'xml')
    for node in soup.find_all('div', {"type":"chapter", "subtype":"source"}):
        for p in node.find_all(['p', 'head']) or p in node_fi:
            text.append(p.text)
    return text

Se på en og en fil i første omgang, for å ut teksten

In [126]:
def make_corpus(zipfolder, extraction = extract_text):
    texts = dict()
    for f in zipfiles:
        texts[f] = extraction(f,zipfolder)
    return texts

In [17]:
def make_dtm(texts):
    dtm = pd.DataFrame()
    freqs = dict()
    for text in texts.keys():
        print(text)
        c = Counter()
        for p in texts[text]:
            c.update(p)
        freqs[text] = nb.frame(c, text)
    dtm = pd.concat([freqs[text] for text in freqs.keys()], axis=1, sort=False)
    return dtm

In [18]:
def sublist(phrase, alist):
    if phrase == [] or alist == []:
        return None
    found = None
    try:
        ix = alist.index(phrase[0])
        searching = True
        while ix < len(alist) and searching == True:
            if alist[ix: ix + len(phrase)] == phrase:
                if found == None:
                    found = [ix]
                else:
                    found.append(ix)
                try:
                    sublist = alist[ix + len(phrase):]
                    ix += sublist.index(phrase[0])
                except:
                    searching = False
            else:
                ix += 1
    except:
        True
    return found

In [40]:
def konk0(phrase, corpus):    
    return [[p,sublist(phrase, p)] for t in corpus for p in t ]

In [316]:
def coll(phrase, corpus, before = 5, after = 5):
    """Determine collocation within corpus if before and after are both zero the whole paragraph is returned"""
    
    collocation = Counter()
    all_of_paragraph = before == 0 and after == 0
    
    for t in corpus:
        for avsnitt in corpus[t]:
            res = sublist(phrase, avsnitt)
            if res != None:
                for i in res:
                    print(i, len(phrase), avsnitt[max(i - before - 1, 0): i + len(phrase) + 1 + after])
                    collocation.update(avsnitt[max(i - before - 1, 0): i + len(phrase) + 1 + after])
    return collocation



In [122]:
def konk(phrase, corpus):
    finds = dict()
    # for hver tekst i korpuset
    for t in corpus:
        finds[t] = []
        # hent avsnittene i korpuset
        for i, avsnitt in enumerate(corpus[t]):
            res = sublist(phrase, avsnitt)
            if res != None:
                finds[t].append((i, res))
    return finds



In [132]:
import re

In [276]:
sublist(['hallo'], "der hallo er hallo der".split())

[1, 3]

In [298]:
def konk_regex(regex, korpus, before=30, after=30, colour='red'):
    res = []
    for t in korpus:
        found = []
        for i, avsnitt in enumerate(korpus[t]):

            s = re.search(regex, avsnitt)
            if s != None:
                p1 = s.span()[0]
                p2 = s.span()[1]
                found.append(str(i) + ": " + avsnitt.replace(avsnitt[p1:p2], "<span style='color:{c}'>".format(c=colour) + avsnitt[p1:p2] + "</span>")[max(0,p1 - before): p2 + after + 31])
        if found != []:
            res.append("#### " + t)
            res += found
    return res

In [315]:
for i in konk_regex("hennes", kilder_korpus_string, before=50, after=50, colour="#22F"):
    display(Markdown(i))

#### NBkilder_Collett_Brev_1841-51.xml

5: ter Jonas Collett nevner «Mad. Braae» og familien <span style='color:#22F'>hennes</span> i brev fra 1839 og 1840 (Collett og Collett 1926

114: – at BraasBraas] trolig Louise Braae (født 1788), <span style='color:#22F'>hennes</span> datter Christiane (Jane) og svigersønn Ernst Fer

206: gusta) (1811–72), Peter Jonas Colletts søster, og <span style='color:#22F'>hennes</span> mann Peter Severin Paulsen Steenstrup (1807–63) 

326: gusta) (1811–72), Peter Jonas Colletts søster, og <span style='color:#22F'>hennes</span> mann Peter Severin Paulsen Steenstrup (1807–63) 

...

901:  kryss eller et kors etter datoen, 3.11.1808 døde <span style='color:#22F'>hennes</span> sønn Wilhelm. Neppe, I Elskede, er der Een iblan

#### NBkilder_Koren_1811.xml

165: e jeg holdt mere af Landet end Byen» – Da det var <span style='color:#22F'>hennes</span> første Samtale med min stille Maja, vidnede det 

#### NBkilder_Undset_Fragmenter.xml

107: det er ikke brukt i de andre fragmentene), også i <span style='color:#22F'>hennes</span> egenhendige trykkmanuskript til Den lykkelige al

In [317]:
coll(['hennes'], kilder_korpus).most_common()

640 1 ['Mad', '.', 'Braae', '»', 'og', 'familien', 'hennes', 'i', 'brev', 'fra', '1839', 'og', '1840']
266 1 ['Braae', '(', 'født', '1788', ')', ',', 'hennes', 'datter', 'Christiane', '(', 'Jane', ')', 'og']
292 1 ['Mad', '.', 'Braae', '»', 'og', 'familien', 'hennes', 'i', 'brev', 'fra', '1839', 'og', '1840']
536 1 ['Peter', 'Jonas', 'Colletts', 'søster', ',', 'og', 'hennes', 'mann', 'Peter', 'Severin', 'Paulsen', 'Steenstrup', '(']
58 1 ['Peter', 'Jonas', 'Colletts', 'søster', ',', 'og', 'hennes', 'mann', 'Peter', 'Severin', 'Paulsen', 'Steenstrup', '(']
19 1 ['Peter', 'Jonas', 'Colletts', 'søster', ',', 'og', 'hennes', 'mann', 'Peter', 'Severin', 'Paulsen', 'Steenstrup', '(']
163 1 ['erfare', ';', 'jeg', 'ankom', 'og', 'forlot', 'hennes', 'bolig', 'på', 'tidspunkter', 'som', 'var', 'for']
57 1 ['frukten', 'av', 'en', 'kjærlighetsaffære', '.', 'Ektemannen', 'hennes', 'er', 'en', 'gammel', 'skravlekopp', '.']
350 1 ['i', 'skade', 'for', 'å', 'få', 'tjenestepiken', 'hennes', 'til', 'å',

[('hennes', 36),
 (',', 27),
 ('og', 20),
 ('.', 15),
 ('en', 10),
 ('i', 9),
 ('på', 7),
 ('Peter', 6),
 ('(', 5),
 ('jeg', 5),
 ('som', 5),
 ('var', 5),
 ('å', 5),
 ('til', 5),
 (')', 4),
 ('søster', 4),
 (';', 4),
 ('to', 4),
 ('om', 4),
 ('kvinne', 4),
 ('med', 4),
 ('Braae', 3),
 ('»', 3),
 ('brev', 3),
 ('fra', 3),
 ('Jonas', 3),
 ('Colletts', 3),
 ('mann', 3),
 ('Severin', 3),
 ('Paulsen', 3),
 ('Steenstrup', 3),
 ('for', 3),
 ('av', 3),
 ('er', 3),
 ('–', 3),
 ('store', 3),
 ('hadde', 3),
 ('ung', 3),
 ('det', 3),
 ('de', 3),
 ('så', 3),
 ('Mad', 2),
 ('familien', 2),
 ('1839', 2),
 ('1840', 2),
 ('datter', 2),
 ('ankom', 2),
 ('få', 2),
 ('hun', 2),
 ('mens', 2),
 ('ut', 2),
 ('ble', 2),
 ('sto', 2),
 ('også', 2),
 ('Da', 2),
 ('fruen', 2),
 ('Den', 2),
 ('fru', 2),
 ('vakre', 2),
 ('kors', 2),
 ('etter', 2),
 ('datoen', 2),
 ('døde', 2),
 ('født', 1),
 ('1788', 1),
 ('Christiane', 1),
 ('Jane', 1),
 ('erfare', 1),
 ('forlot', 1),
 ('bolig', 1),
 ('tidspunkter', 1),
 ('frukten

In [235]:
a ="det er er elvete"
s = re.search('elv', a)
a[s.span()[0] - 3:s.span()[1] + 2]

'er elvet'

In [141]:
found

[]

In [23]:
kildezip = "XML_NBkilder_20191112.zip"

In [24]:
kildefiler = get_files_from_zip(kildezip)

In [25]:
kilder_korpus = make_corpus(kildezip)

In [127]:
kilder_korpus_string = make_corpus(kildezip, extraction= extract_text_as_string)

In [128]:
kilder_korpus_string['NBkilder_Amundsen_Slededagbok.xml']

['[september 1911]',
 'Freda – 8 septbr.',
 'Endeli bar de aste idag – menn ikke uten vanskliheterr. Da vi hadde spennt fårr alle sleerne sprang HHsHH] Helmer Hanssen (1870-1956) & WsW] Oscar Wisting (1870-1936) hunner ut me denn følge, att di måtte springe efter demm en lang strekkning, innenn di nådde demm ijenn. Førsst kl. ½ 1 emd. kunne vi sette ut fra startplassenn. Føre har værrt glimrenne idag. Sjeldenn har jei kjennt føre så gått. Sell leder jei menns di anndre føllerr effter me værrt sitt kåbbell. Ståppete kl ½ 4 emd effterå ha jorrt 10.4 kvmkvm] kvartmil = 1 nautisk mil = 1852 m fra FramheimFramheim] Ekspedisjonens vinterkvarter. Desværre har 3 a Kamillas onger fullt me å di må slaktess snaresst. Være var hærrli innåver hær. Stille – dels klart ⁒38°. Vi ær kåmmett litt fårr langt W liW li] vestlig å må imåren hålle E liere bare fårrå kåmme åpp i gamle veienn vår ijenn.',
 'Lørda – 9 septbr.',
 "Aste kl. ½ 8 fmd effter frykteli natt. «Kaisa» – en tispe såmm tillhører Pr. «flyr

In [29]:
kilder_dtm = make_dtm(kilder_korpus).fillna(0)

NBkilder_Amundsen_Slededagbok.xml
NBkilder_Collett_Brev_1841-51.xml
NBkilder_Collett_Brev_til_Bjornson.xml
NBkilder_Collett_Brev_til_Ibsen.xml
NBkilder_Collett_Brev_til_Lie.xml
NBkilder_Collett_Skildpadde.xml
NBkilder_Dardel_Voyage.xml
NBkilder_Grunnlovskilder1814.xml
NBkilder_Kjerulf_1833.xml
NBkilder_Kjerulf_1840.xml
NBkilder_Kjerulf_1850.xml
NBkilder_Kjerulf_1851.xml
NBkilder_Kjerulf_1852.xml
NBkilder_KKielland_AGarborg_brev.xml
NBkilder_KKielland_DSkavlan_brev.xml
NBkilder_KKielland_EPeterssen_brev.xml
NBkilder_Koren_1798.xml
NBkilder_Koren_1805.xml
NBkilder_Koren_1808.xml
NBkilder_Koren_1809.xml
NBkilder_Koren_1810.xml
NBkilder_Koren_1811.xml
NBkilder_Koren_1812.xml
NBkilder_Koren_1813.xml
NBkilder_Koren_1814.xml
NBkilder_Koren_1815.xml
NBkilder_Undset_Fragmenter.xml


In [26]:
kildefiler

['NBkilder_Amundsen_Slededagbok.xml',
 'NBkilder_Collett_Brev_1841-51.xml',
 'NBkilder_Collett_Brev_til_Bjornson.xml',
 'NBkilder_Collett_Brev_til_Ibsen.xml',
 'NBkilder_Collett_Brev_til_Lie.xml',
 'NBkilder_Collett_Skildpadde.xml',
 'NBkilder_Dardel_Voyage.xml',
 'NBkilder_Grunnlovskilder1814.xml',
 'NBkilder_Kjerulf_1833.xml',
 'NBkilder_Kjerulf_1840.xml',
 'NBkilder_Kjerulf_1850.xml',
 'NBkilder_Kjerulf_1851.xml',
 'NBkilder_Kjerulf_1852.xml',
 'NBkilder_KKielland_AGarborg_brev.xml',
 'NBkilder_KKielland_DSkavlan_brev.xml',
 'NBkilder_KKielland_EPeterssen_brev.xml',
 'NBkilder_Koren_1798.xml',
 'NBkilder_Koren_1805.xml',
 'NBkilder_Koren_1808.xml',
 'NBkilder_Koren_1809.xml',
 'NBkilder_Koren_1810.xml',
 'NBkilder_Koren_1811.xml',
 'NBkilder_Koren_1812.xml',
 'NBkilder_Koren_1813.xml',
 'NBkilder_Koren_1814.xml',
 'NBkilder_Koren_1815.xml',
 'NBkilder_Undset_Fragmenter.xml']

In [124]:
kilder_dtm.sum().sum()

1386339.0

In [30]:
nb.frame_sort(kilder_dtm, 'NBkilder_Koren_1815.xml').head(50).style.background_gradient()

In [38]:
print('\n'.join([' '.join(y) for y in kilder_korpus['NBkilder_Collett_Brev_til_Bjornson.xml']]))

4. september 1867
Kjere Bjørnson .
Da jeg hørte De var i Kjøbenhavn , kom det mig som et Vink fra Oven , at henvende mig til Dem .
Kan De ikke redde mig af min Nød ? . . den dobbelte , at se sit aandelige Arbeide spildt og se sig skuffet for den jordiske materielle Løn derfor , hvilken Sidste vi saa gjerne ville ignorere om vi kunde , vilde overse som noget Ringe , indtil vi en vakker Dag mærke at vor hele Existenz afhænger af den , og at vi bedre kunne undvære Æren , end et Par fattige lumpne hundrede Daler .
For at sætte Dem korteligt ind i dette , vil jeg sige Dem , at siden min Søn EmilsEmil Collett ( 1848 – 1904 ) Anbringelse paa fremmet Sted , samtidig med uopfyldte Forpligtelser fra Andres Side ( TønsbergHenning B. Tønsberg ( 1837 – 81 ) samme Høst 120 Spd ) jeg har jeg maattet lære hvad Nød og Forlegenhed , lammet Livsmod og Virksomhed vil sige . Det var blot i gamle Dage at Digterne kunde leve af Maaneskin og Skovluft men de have taget Recepten til denne Kunst med i Graven , t

In [120]:
actual_konk = konk(['anderledes'], kilder_korpus)
actual_konk

{'NBkilder_Amundsen_Slededagbok.xml': [(108, [381]), (219, [112])],
 'NBkilder_Collett_Brev_1841-51.xml': [(186, [187]),
  (247, [20]),
  (270, [901])],
 'NBkilder_Collett_Brev_til_Bjornson.xml': [(16, [54])],
 'NBkilder_Collett_Brev_til_Ibsen.xml': [(102, [32]), (151, [15])],
 'NBkilder_Collett_Brev_til_Lie.xml': [(4, [65]),
  (17, [165]),
  (19, [66]),
  (52, [268])],
 'NBkilder_Collett_Skildpadde.xml': [(27, [850])],
 'NBkilder_Dardel_Voyage.xml': [],
 'NBkilder_Grunnlovskilder1814.xml': [],
 'NBkilder_Kjerulf_1833.xml': [],
 'NBkilder_Kjerulf_1840.xml': [],
 'NBkilder_Kjerulf_1850.xml': [(165, [43]), (246, [716])],
 'NBkilder_Kjerulf_1851.xml': [(69, [290])],
 'NBkilder_Kjerulf_1852.xml': [],
 'NBkilder_KKielland_AGarborg_brev.xml': [(50, [47]),
  (72, [721]),
  (123, [118]),
  (215, [850]),
  (275, [66]),
  (309, [135]),
  (334, [52]),
  (432, [108])],
 'NBkilder_KKielland_DSkavlan_brev.xml': [(102, [513])],
 'NBkilder_KKielland_EPeterssen_brev.xml': [(68, [273]),
  (114, [470, 52

In [64]:
from IPython.display import display, Markdown

In [111]:
def korpus_konk(phrase, korpus, before = 10, after = 10):
    actual_konk = konk(phrase, korpus)
    result = ["### Konkordanser for *" + ' '.join(phrase) + "*" ]
    for i in actual_konk:
        if actual_konk[i] != []:
            header = "#### " + str(i)
            result.append(header)
            for hit in actual_konk[i]:
                para = hit[0]
                parahits = hit[1]
                konkstring = []
                for y in parahits:
                    konkstring.append(str(para) + ": " + ' '.join(korpus[i][para][y - before : y+after]))
                result += konkstring
    return result

In [121]:
for i in korpus_konk(['jeg', 'er'], kilder_korpus, before=8, after=8):
    try:
        display(Markdown(i))
    except:
        print(i)

### Konkordanser for *jeg er*

#### NBkilder_Collett_Brev_1841-51.xml

6: at han er saa lykkelig , og at jeg er saa elskværdig mod ham . Jeg

6: saa elskværdig mod ham . Jeg troer ogsaa jeg er det , men hvorledes skulde det

82: troe ikke jeg knurrer engang – men naar jeg er færdig er jeg erschöpft , erschöpft

...

169: , der ikke vet , hvor bundløst rik jeg er – det er bare som en

171: mig til mit arbeide ? Jeg som synes jeg er kommet utenfor livet – det som

171: er der , men jeg husker , hvor jeg er født og baaren og hvor min

171: kom dit . Og sommetider , netop naar jeg er mest lykkelig , saa stikker det

207: tror ikke jeg kunde blit saa lykkelig som jeg er , hvis der ikke hadde været

In [84]:
display(Markdown("## Hallo der"))
display(Markdown("<br>"))
display(Markdown('der'))

## Hallo der

<br>

der